In [1]:
import riotwatcher as rw
import dotenv
import os
import arrow
import aiolimiter
import asyncio
import jsonlines
import time
import aiohttp
import pyrate_limiter
import zlib
import gzip
import tenacity
import aiofiles
import random
import json
import sys

In [2]:
dotenv.load_dotenv()

True

In [3]:
patch_date_range = {'start':arrow.get(2023, 6, 1, 0,0,0).int_timestamp, 'end':arrow.get(2023, 6, 14, 0,0,0).int_timestamp} # Patch 13.11

In [ ]:
lol_watcher = rw.LolWatcher(os.environ.get("RIOT_API_KEY"))

In [ ]:
lol_watcher.summoner.by_name(summoner_name="Cigarrrr", region="PH2")

In [ ]:
class DataExtract1:
    def __init__(self, seed, num_dp, patch_date, file_name):
        self.limiter = aiolimiter.AsyncLimiter(max_rate=100, time_period=120)
        self.data_counter = 0
        self.num_dp = num_dp
        self.patch_date = patch_date
        self.seed_players = [seed]
        self.done_players = set()
        self.matches_id = []
        self.done_matches = set()
        self.matches_json = []
        self.file_name = file_name
        with jsonlines.open(f'{self.file_name}.ndjson', mode='w') as out:
            pass
    
    async def get_matches(self, seed_player):
        async with self.limiter:
            return lol_watcher.match.matchlist_by_puuid(puuid=seed_player, region="PH2", count=100, queue=450, start_time=self.patch_date['start'], end_time=self.patch_date['end'])

    async def get_match(self, seed_match):
        async with self.limiter:
            return lol_watcher.match.by_id(match_id=seed_match, region="PH2")

    async def get_player(self, seed_player):
        async with self.limiter:
            return lol_watcher.summoner.by_puuid(encrypted_puuid=seed_player, region="PH2")

    def extract_players(self, seed_match_json):
        return [player for player in seed_match_json['metadata']['participants']]
    
    def flatten(self, to_flatten, level):
        match level:
            case 2:
                return [x for xs in to_flatten for x in xs]
            case 3:
                return [x for xs in to_flatten for xs2 in xs for x in xs2]

    async def main(self):
        while self.data_counter < self.num_dp:
            matchlist_tasks = [self.get_matches(seed_player) for seed_player in self.seed_players if (seed_player not in self.done_players)]
            matchlist_response = await asyncio.gather(*matchlist_tasks)
            self.matches_id.append(matchlist_response)
            self.matches_id = self.flatten(self.matches_id, 3)

            for seed in self.seed_players:
                self.done_players.add(seed)

            match_tasks = [self.get_match(seed_match) for seed_match in self.matches_id if (seed_match not in self.done_matches)]
            match_response = await asyncio.gather(*match_tasks)
            self.matches_json.append(match_response)
            self.matches_json = self.flatten(self.matches_json, 2)
            with jsonlines.open(f'{self.file_name}.ndjson', mode='a') as out:
                for match in self.matches_json:
                    out.write(match)

            self.seed_players = [participant for seed_match_json in self.matches_json for participant in seed_match_json['metadata']['participants'] if seed_match_json['metadata']['matchId'] not in self.done_matches]

            for match in self.matches_id:
                self.done_matches.add(match)
            self.matches_id = []
            self.matches_json = []

            self.data_counter = len(self.done_matches)
            print(self.data_counter)

In [ ]:
extractor1 = DataExtract(seed="uRZ9d8VNRDsYFDtlTy3Cr9VTa3rFN3Y504Woem4uyYdCGWZsw2uKF9OoX3DbiseOXOuq7dajnxamtw",
                         num_dp=200, patch_date=patch_date_range, file_name='test1')

In [ ]:
await extractor1.main()

In [ ]:
class SyncDataExtract:
    def __init__(self, seed, num_dp, patch_date, file_name):
        self.data_counter = 0
        self.num_dp = num_dp
        self.patch_date = patch_date
        self.seed_players = [seed]
        self.done_players = set()
        self.matches_id = []
        self.done_matches = set()
        self.matches_json = []
        self.file_name = file_name
        with jsonlines.open(f'{self.file_name}.ndjson', mode='w') as out:
            pass
    
    def get_matches(self, seed_player):
        return lol_watcher.match.matchlist_by_puuid(puuid=seed_player, region="PH2", count=100, queue=450, start_time=self.patch_date['start'], end_time=self.patch_date['end'])

    def get_match(self, seed_match):
        return lol_watcher.match.by_id(match_id=seed_match, region="PH2")

    def get_player(self, seed_player):
        return lol_watcher.summoner.by_puuid(encrypted_puuid=seed_player, region="PH2")

    def extract_players(self, seed_match_json):
        return [player for player in seed_match_json['metadata']['participants']]
    
    def flatten(self, to_flatten, level):
        match level:
            case 2:
                return [x for xs in to_flatten for x in xs]
            case 3:
                return [x for xs in to_flatten for xs2 in xs for x in xs2]

    def main(self):
        while self.data_counter < self.num_dp:
            for seed_player in self.seed_players:
                matchlist_response = self.get_matches(seed_player)
                time.sleep(1.2)
                self.matches_id.append(matchlist_response)
            self.matches_id = self.flatten(self.matches_id, 2)

            for seed in self.seed_players:
                self.done_players.add(seed)
                
            for match_id in self.matches_id:
                match_response = self.get_match(match_id)
                time.sleep(1.2)
                self.matches_json.append(match_response)
            # self.matches_json = self.flatten(self.matches_json, 2)
            with jsonlines.open(f'{self.file_name}.ndjson', mode='a') as out:
                for match in self.matches_json:
                    out.write(match)

            self.seed_players = [participant for seed_match_json in self.matches_json for participant in seed_match_json['metadata']['participants'] if seed_match_json['metadata']['matchId'] not in self.done_matches]

            for match in self.matches_id:
                self.done_matches.add(match)
            self.matches_id = []
            self.matches_json = []

            self.data_counter = len(self.done_matches)
            print(self.data_counter)

In [ ]:
extractor2 = SyncDataExtract(seed="uRZ9d8VNRDsYFDtlTy3Cr9VTa3rFN3Y504Woem4uyYdCGWZsw2uKF9OoX3DbiseOXOuq7dajnxamtw",
                         num_dp=200, patch_date=patch_date_range, file_name='test2')

In [ ]:
%%time
extractor2.main()

In [ ]:
class AsyncDataExtract2:
    def __init__(self, headers, seed, num_dp, patch_date, file_name):
        self.rate_limits = (
            pyrate_limiter.RequestRate(20, 1),
            pyrate_limiter.RequestRate(100, 120)
        )
        self.limiter = pyrate_limiter.Limiter(*self.rate_limits)
        self.client = aiohttp.ClientSession(headers=headers)
        self.data_counter = 0
        self.num_dp = num_dp
        self.patch_date = patch_date
        self.seed_players = [seed]
        self.done_players = set()
        self.matches_id = []
        self.done_matches = set()
        self.matches_json = []
        self.file_name = file_name
        with jsonlines.open(f'{self.file_name}.jsonl', 'w') as out:
            pass
    
    def is_exceed_response(response):
        return 'metadata' not in response
    
    def flatten(self, to_flatten, level):
        match level:
            case 2:
                return [x for xs in to_flatten for x in xs]
            case 3:
                return [x for xs in to_flatten for xs2 in xs for x in xs2]
    
    def extract_players(self, seed_match_json):
        return [player for player in seed_match_json['metadata']['participants']]
    
    async def get_matches(self, seed_player):
        async with self.limiter.ratelimit('get_matches', delay=True):
            response = await self.client.get(f'https://sea.api.riotgames.com/lol/match/v5/matches/by-puuid/{seed_player}/ids?startTime={self.patch_date["start"]}&endTime={self.patch_date["end"]}&queue=450&start=0&count=100')
            return await response.json()

    @retry(retry=retry_if_result(is_exceed_response))
    async def get_match(self, seed_match):
        async with self.limiter.ratelimit('get_match', delay=True):
            response = await self.client.get(f'https://sea.api.riotgames.com/lol/match/v5/matches/{seed_match}')
            response_json = await response.json()
            return response_json

    async def get_player_rank(self, seed_player_id):
        async with self.limiter.ratelimit('get_matches', delay=True):
            pass
    
    async def gather_matches(self):
        while self.data_counter < self.num_dp:
            matchlist_tasks = [self.get_matches(seed_player) for seed_player in self.seed_players if (seed_player not in self.done_players)]
            matchlist_response = await asyncio.gather(*matchlist_tasks)
            self.matches_id.append(matchlist_response)
            self.matches_id = self.flatten(self.matches_id, 3)

            for seed in self.seed_players:
                self.done_players.add(seed)

            match_tasks = [self.get_match(seed_match) for seed_match in self.matches_id if (seed_match not in self.done_matches)]
            self.matches_json.append(await asyncio.gather(*match_tasks))
            self.matches_json = self.flatten(self.matches_json, 2)
            
            with jsonlines.open(f'{self.file_name}.jsonl', 'a') as out:
                for match in self.matches_json:
                    out.write(match)

            self.seed_players = [participant for seed_match_json in self.matches_json for participant in seed_match_json['metadata']['participants'] if seed_match_json['metadata']['matchId'] not in self.done_matches]

            for match in self.matches_id:
                self.done_matches.add(match)
            self.matches_id = []
            self.matches_json = []

            self.data_counter = len(self.done_matches)
            print(self.data_counter)
        
        await self.client.close()

In [ ]:
extractor3 = DataExtract(headers=headers, seed="uRZ9d8VNRDsYFDtlTy3Cr9VTa3rFN3Y504Woem4uyYdCGWZsw2uKF9OoX3DbiseOXOuq7dajnxamtw",
                         num_dp=100000, patch_date=patch_date_range, file_name='test3')

In [ ]:
await extractor3.gather_matches()

In [4]:
class AsyncDataExtract3:
    def __init__(self, headers, seed, num_dp, patch_date, file_name):
        self.limiter = aiolimiter.AsyncLimiter(max_rate=1, time_period=1.2)
        self.client = aiohttp.ClientSession(headers=headers)
        self.data_counter = 0
        self.num_dp = num_dp
        self.patch_date = patch_date
        self.seed_players = [seed]
        self.done_players = set()
        self.done_matches = set()
        self.file_name = file_name
        
    def __del__(self):
        self.client.close()
    
    def flatten(self, to_flatten, level):
        match level:
            case 2:
                return [x for xs in to_flatten for x in xs]
            case 3:
                return [x for xs in to_flatten for xs2 in xs for x in xs2]
    
    def extract_players(self, seed_match_json):
        return [player for player in seed_match_json['metadata']['participants']]
    
    def no_connection_warning(self, err):
        print('No connection or exceeded requests!')
    
    async def get_matches(self, seed_player):
        async for attempt in tenacity.AsyncRetrying(wait=tenacity.wait_fixed(10000), retry=tenacity.retry_if_exception_type((aiohttp.ClientConnectorError, aiohttp.ClientOSError, aiohttp.ClientResponseError)), after=self.no_connection_warning):
            with attempt:
                async with self.limiter:
                    response = await self.client.get(f'https://sea.api.riotgames.com/lol/match/v5/matches/by-puuid/{seed_player}/ids?startTime={self.patch_date["start"]}&endTime={self.patch_date["end"]}&queue=450&start=0&count=100')
                    response_json = await response.json()
                if ('status' in response_json) and (response_json['status']['status_code'] == 429):
                    raise aiohttp.ClientResponseError(request_info=response.request_info, history=response.history, status=429)
            if not attempt.retry_state.outcome.failed:
                attempt.retry_state.set_result(response_json)
        return response_json

    async def get_match(self, seed_match):
        async for attempt in tenacity.AsyncRetrying(wait=tenacity.wait_fixed(10000), retry=tenacity.retry_if_exception_type((aiohttp.ClientConnectorError, aiohttp.ClientOSError, aiohttp.ClientResponseError)), after=self.no_connection_warning):
            with attempt:
                async with self.limiter:
                    response = await self.client.get(f'https://sea.api.riotgames.com/lol/match/v5/matches/{seed_match}')
                    response_json = await response.json()
                if ('status' in response_json) and (response_json['status']['status_code'] == 429):
                    raise aiohttp.ClientResponseError(request_info=response.request_info, history=response.history, status=429)
            if not attempt.retry_state.outcome.failed:
                attempt.retry_state.set_result(response_json)
        return response_json
    
    async def gather_matches(self):
        with gzip.open(f'{self.file_name}.jsonl.gz', mode='ab') as out_file:
            while self.data_counter < self.num_dp:
                seed_player = random.choice(self.seed_players)
                self.seed_players.remove(seed_player)
                if seed_player not in self.done_players:
                    matchlist_response = await self.get_matches(seed_player)
                    self.done_players.add(seed_player)
                    
                    if not matchlist_response: continue
                    match_tasks = [self.get_match(seed_match) for seed_match in matchlist_response if seed_match not in self.done_matches]
                    if not match_tasks: continue
                    matches_json = await asyncio.gather(*match_tasks)

                    for seed_match_json in matches_json:
                        if 'metadata' in seed_match_json:
                            for participant in seed_match_json['metadata']['participants']:
                                if participant not in self.done_players:
                                    self.seed_players.append(participant)
                                    
                            json_buffer = json.dumps(seed_match_json).encode('utf-8')
                            out_file.write(json_buffer)
                            self.done_matches.add(seed_match_json['metadata']['matchId'])

                    self.data_counter = len(self.done_matches)
                    print(self.data_counter)
        
        await self.client.close()

In [5]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": os.environ.get('RIOT_API_KEY')
}

In [6]:
extractor4 = AsyncDataExtract3(headers=headers, seed='rnSanmBvzma5coD59loD39if9nkV3Yo28i4ool_8Sa83m38n-EGDzt1qGUCs33lPxA1dvuyGCwBWRA',
                               num_dp=10000, patch_date=patch_date_range, file_name='test4')

In [7]:
await extractor4.gather_matches()

1
37
66
92
182
241
273
280
312
338
373
465
466
534
594
619
648
727
771
798
856
924
938
941
993
1016
1022
1121
1164
1173
1196
1255
1287
1382
1425
1462
1529
1566
1612
1699
1733
1762
1777
1786
1793
1842
1862
1924
1979
1992
1998
2008
2030
2129
2226
2236
2240
2266
2279
2311
2314
2362
2408
2461
2560
2591
2624
2693
2762
2767
2773
2789
2797
2848
2908
2955
2982
2996
3023
3066
3067
3155
3164
3175
3193
3235
3252
3274
3338
3412
3479
3499
3514
3547
3597
3636
3648
3687
3725
3759
3766
3772
3818
3857
3900
3947
3978
4022
4055
4072
4159
4249
4268
4291
4336
4405
4470
4473
4545
4557
4590
4649
4676
4698
4736
4796
4806
4848
4870
4966
4999
5078
5117
5132
5199
5207
5277
5312
5358
5370
5387
5420
5441
5488
5497
5537
5552
5594
5602
5615
5656
5663
5680
5687
5713
5778
5807
5819
5826
5827
5846
5863
5897
5979
6005
6013
6070
6127
6152
6170
6189
6234
6261
6308
6354
6404
6460
6521
6547
6584
6630
6645
6656
6721
6777
6820
6858
6933
6972
7009
7024
7074
7118
7120
7151
7167
7170
7188
7252
7292
7322
7402
7433
7491
7506
7592


In [ ]:
class AsyncDataExtract4:
    def __init__(self, headers, seed, num_dp, patch_date, file_name):
        self.rate_limits = pyrate_limiter.RequestRate(100, 120)
        self.limiter = pyrate_limiter.Limiter(self.rate_limits)
        self.client = aiohttp.ClientSession(headers=headers)
        self.data_counter = 0
        self.num_dp = num_dp
        self.patch_date = patch_date
        self.seed_players = [seed]
        self.done_players = set()
        self.matches_id = []
        self.done_matches = set()
        self.matches_json = []
        self.file_name = file_name
        with jsonlines.open(f'{self.file_name}.jsonl', 'w') as out:
            pass
    
    def is_exceed_response(self, response):
        print('exceeded')
        return 'metadata' not in response
    
    def flatten(self, to_flatten, level):
        match level:
            case 2:
                return [x for xs in to_flatten for x in xs]
            case 3:
                return [x for xs in to_flatten for xs2 in xs for x in xs2]
    
    def extract_players(self, seed_match_json):
        return [player for player in seed_match_json['metadata']['participants']]
    
    async def get_matches(self, seed_player):
        async with self.limiter.ratelimit('get_matches', delay=True):
            response = await self.client.get(f'https://sea.api.riotgames.com/lol/match/v5/matches/by-puuid/{seed_player}/ids?startTime={self.patch_date["start"]}&endTime={self.patch_date["end"]}&queue=450&start=0&count=100')
            return await response.json()

    async def get_match(self, seed_match):
        try:
            async for attempt in tenacity.AsyncRetrying(wait=tenacity.wait_fixed(1)+tenacity.wait_random(0,1), stop=tenacity.stop_after_attempt(3), reraise=True): # retry=tenacity.retry_if_exception_type(aiohttp.ClientResponseError)
                with attempt:
                    async with self.limiter.ratelimit('get_match', delay=True):
                        response = await self.client.get(f'https://sea.api.riotgames.com/lol/match/v5/matches/{seed_match}')
                        response_json = await response.json()
                    if ('status' in response_json) and (response_json['status']['status_code'] == 429):
                        raise aiohttp.ClientResponseError(request_info=response.request_info, history=response.history, status=429)
                if not attempt.retry_state.outcome.failed:
                    attempt.retry_state.set_result(response_json)
            return response_json
        except aiohttp.ClientResponseError:
            print('Exceeded rate')

    async def get_player_rank(self, seed_player_id):
        async with self.limiter.ratelimit('get_matches', delay=True):
            pass
    
    async def gather_matches(self):
        while self.data_counter < self.num_dp:
            seed_player = self.seed_players.pop()
            if seed_player not in self.done_players:
                matchlist_response = await self.get_matches(seed_player)
                self.matches_id.append(matchlist_response)
                self.matches_id = self.flatten(self.matches_id, 2)

                self.done_players.add(seed_player)

                match_tasks = [self.get_match(seed_match) for seed_match in self.matches_id if seed_match not in self.done_matches]
                self.matches_json.append(await asyncio.gather(*match_tasks))
                self.matches_json = self.flatten(self.matches_json, 2)
                with jsonlines.open(f'{self.file_name}.jsonl', 'a') as out:
                    for match in self.matches_json:
                        out.write(match)

                self.seed_players.append([participant for seed_match_json in self.matches_json for participant in seed_match_json['metadata']['participants'] if participant not in self.done_players])
                self.seed_players = self.flatten(self.seed_players, 2)

                for match in self.matches_json:
                    self.done_matches.add(match['metadata']['matchId'])
                self.matches_id = []
                self.matches_json = []

                self.data_counter = len(self.done_matches)
                print(self.data_counter)
        
        await self.client.close()

## Results

<pre>Synchronous
CPU times: total: 547 ms
Wall time: 7min 57s

Asynchronous - dripping limiter (AsyncDataExtract1)
Wall time: 7min 41s

Asynchronous - burst limiter (AsyncDataExtract1)
Wall time: 6min 3s

Asynchronous - aiohttp and pyrate-limiter (AsyncDataExtract2)
Wall time: 6min 1s

Asynchronous - cut down on async gather tasks (AsyncDataExtract4)
Wall time: 4min (200 num_dp)

Asynchronous - aiolimiter dripping limiter (AsyncDataExtract3) <-- chosen one
Wall time: 4min 37s</pre>

Note to self: whole code works up until file writing, code infinitely loops which means either self.done_matches is
not updating or there is some error in rate limiting or async is just not working

Note to self: 6/22/2023 i made changes (specifically removing await in getter functions and put await gather tasks in variable)
still the same...

Note to self: 6/24/2023 i just went with manual way of flattening (rather than using yield). Would be interesting to know more
in the future why the semantic error from before happens

Note to self: 6/25/2023 async code is done and working now just need to benchmark with sync code;
done benchmarking, everything's good